<a href="https://colab.research.google.com/github/Apekshaa2908/Enhancing-E-Commerce-Agility-With-Advanced-ETL-Pipeline/blob/main/Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit
!pip install boto3

In [ ]:
%%writefile app.py
import streamlit as st
import boto3
from botocore.exceptions import NoCredentialsError, ClientError
import json

def get_aws_credentials(secret_name):
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(service_name='secretsmanager', region_name='ap-southeast-2')

    try:
        # Retrieve the secret
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
        secret = get_secret_value_response['SecretString']
        return json.loads(secret)  # Return as dictionary
    except ClientError as e:
        st.error(f"Error retrieving AWS credentials: {e}")
        return None

def upload_to_s3(file, bucket, filename):
    # Get AWS credentials from Secrets Manager
    credentials = get_aws_credentials("AWS_CREDENTIALS")
    if credentials is None:
        return

    # Provide AWS credentials explicitly
    s3 = boto3.client(
        's3',
        aws_access_key_id=credentials['aws_access_key_id'],
        aws_secret_access_key=credentials['aws_secret_access_key'],
        region_name='ap-southeast-2'
    )

    try:
        s3.upload_fileobj(file, bucket, filename)
        st.success(f'File {filename} uploaded successfully to {bucket}')
    except NoCredentialsError:
        st.error('Credentials not available')

st.title('Order and Returns Data Upload')

uploaded_file = st.file_uploader("Choose a file")

if uploaded_file:
    st.write(f"Selected file: {uploaded_file.name}")

    # Create a button to trigger the upload
    if st.button("Upload File"):
        if 'order table' in uploaded_file.name.lower():
            upload_to_s3(uploaded_file, 'order-apekshaa', uploaded_file.name)
        elif 'return table' in uploaded_file.name.lower():
            upload_to_s3(uploaded_file, 'return-apekshaa', uploaded_file.name)
        else:
            st.error("Invalid file. Please upload either an 'order table' or 'Return Table' file.")

In [ ]:
! wget -q -O - ipv4.icanhazip.com
! streamlit run app.py & npx localtunnel --port 8501